In [1]:
import numpy as np
import pandas as pd
import math
import random

# Support

In [2]:
def count_label(data):
    distinct = [data[0][-1]]
    for i in range(len(data)):
        check = True
        for j in range(len(distinct)):
            if distinct[j] == data[i][-1]:
                check = False
        if check:
            distinct.append(data[i][-1])
    return np.array(distinct).astype(int), len(distinct)

In [3]:
def evaluation(arr_acc):
    mean = np.mean(arr_acc)
    varian = sum((arr_acc - mean) * (arr_acc - mean)) / len(arr_acc)
    std = math.sqrt(varian)
    return mean, std

In [5]:
def euclidian(X, Y):
    euc = 0
    for i in range(len(X)):
        euc += math.pow((X[i]-Y[i]), 2)
    euc = math.sqrt(euc)
    return euc

# Pre Processing

In [6]:
def randomize(arr):
    for i in range(len(arr)-1, 0, -1):
        j = random.randint(0, i)
        arr[i], arr[j] = arr[j], arr[i]
    return arr

In [7]:
def normalize(data):
    data = np.array(data).astype(float)
    max_value = np.max(data, axis=0)
    min_value = np.min(data, axis=0)
    for i in range(len(data)):
        for j in range(len(data[0])-1):
            data[i][j] = (data[i][j] - min_value[j]) / (max_value[j] - min_value[j])
    return data

In [8]:
def handleMissValue(data):
    filled_data = np.array([[0 for i in range(len(data[0]))] for j in range(len(data))])
    cLabel = count_label(data)[1]
    arr_data = [[] for j in range(cLabel)]
    mean = []
    for i in range(len(data)):
        arr_data[int(data[i][-1])-1].append(data[i])
    counter = 0
    for i in range(len(arr_data)):
        mean.append(np.nanmean(arr_data[i], axis=0))
        for j in range(len(arr_data[i])):
            for k in range(len(arr_data[i][j])):
                if np.isnan(arr_data[i][j][k]):
                    arr_data[i][j][k] = mean[i][k]
            filled_data[counter] = arr_data[i][j]
            counter += 1
    return filled_data

In [9]:
def extraction(file, is_normal = True, is_random = True):
    data = pd.read_excel(file)
    data = handleMissValue(np.array(data))
    data = pd.DataFrame(data)
    data.drop_duplicates()
    data = np.array(data)
    if is_normal:
        data = normalize(data)
    if is_random:
        data = randomize(data.tolist())
    return data

In [10]:
def crossValidation(data, cross_val, cross_index, label_index):
    split_test = int(len(data) * (1 / cross_val))
    change_index = (cross_index - 1) * split_test
    
    for i in range(split_test):
        j = (i + change_index)
        data[i], data[j] = data[j], data[i]
    
    data_train_feature = pd.DataFrame(data[split_test:]).drop([label_index], axis=1)
    data_train_label = pd.DataFrame(pd.DataFrame(data[split_test:]), columns=[label_index]).astype(np.int)
    data_test_feature = pd.DataFrame(data[0:split_test]).drop([label_index], axis=1)
    data_test_label = pd.DataFrame(pd.DataFrame(data[0:split_test]), columns=[label_index]).astype(np.int)
    
    data_train_feature = np.array(data_train_feature)
    data_train_label = np.array(data_train_label)
    data_test_feature = np.array(data_test_feature)
    data_test_label = np.array(data_test_label)

    return data, data_train_feature, data_train_label, data_test_feature, data_test_label

# LMKNN

In [11]:
def closest(Xx, Xx_label, Y, cLabel, k):
    distance = [[None for i in range(len(Xx))] for j in range(cLabel)]
    min_distance_value = []
    min_distance_index = []
    for i in range(len(Xx)):
        distance[((Xx_label[i][0])-1)][i] = euclidian(Xx[i], Y)
    for i in range(cLabel):
        min_distance_value.append([])
        min_distance_index.append([])
        for j in range(k):
            mdv = min(x for x in distance[i] if x not in min_distance_value[i] and x is not None)
            mdi = distance[i].index(min(x for x in distance[i] if x not in min_distance_value[i] and x is not None)) 
            min_distance_value[i].append(mdv)
            min_distance_index[i].append(mdi)
    return distance, min_distance_value, min_distance_index

In [12]:
def localMeanVector(closest, Xx, Y, k):
    local_mean = [0 for i in range(len(closest))]
    for i in range(len(closest)):
        for j in range(len(closest[i])):
            local_mean[i] += Xx[closest[i][j]]
        local_mean[i] = local_mean[i] / k
    return local_mean

In [13]:
def decision(local_mean, Y, Y_label):
    distance = [0 for i in range(len(local_mean))]
    for i in range(len(local_mean)):
        distance[i] = euclidian(local_mean[i], Y)
    argmin = distance.index(min(distance)) + 1
    return argmin, Y_label[0]

# Testing

In [14]:
k = 15
file = 'Dataset Pegawai.xlsx'
cross_val = 10
label_index = 9 #Kinerja
normalisasi = True
random_data = True
data = extraction(file, normalisasi, random_data)
arr_kelas, jml_kelas = count_label(data)

In [ ]:
arr_mean_acc = [0 for i in range(k)]
arr_std_acc  = [0 for i in range(k)]

for a in range(1, k + 1): #Pengujian Parameter K
    arr_acc = [0 for i in range(cross_val)]
    for i in range(1, cross_val + 1):
        acc = 0
        data, train_f, train_l, test_f, test_l = crossValidation(data, cross_val, i, label_index)
        for j in range(len(test_f)):
            distance, min_value, min_index = closest(train_f, train_l, test_f[j], jml_kelas, a)
            local_mean = localMeanVector(min_index, train_f, test_f[j], a)
            predict, actual = decision(local_mean, test_f[j], test_l[j])
            if predict == actual:
                acc += 1
        arr_acc[i-1] = acc/len(test_f)
    mean, std = evaluation(arr_acc)
    arr_mean_acc[a-1] = mean
    arr_std_acc[a-1] = std
    print('K-{0} Mean : {1} with Standar Deviation : {2}.'.format(a,mean,std))

K-1 Mean : 0.9519756838905776 with Standar Deviation : 0.01077206391894791.
K-2 Mean : 0.9591185410334347 with Standar Deviation : 0.009230568169641459.
K-3 Mean : 0.9612462006079026 with Standar Deviation : 0.01044660112831877.
K-4 Mean : 0.9598784194528875 with Standar Deviation : 0.007635171238346798.
K-5 Mean : 0.96048632218845 with Standar Deviation : 0.008861629019521729.
K-6 Mean : 0.9601823708206687 with Standar Deviation : 0.008856414762512744.
K-7 Mean : 0.9617021276595745 with Standar Deviation : 0.00890841999377073.
K-8 Mean : 0.9620060790273556 with Standar Deviation : 0.0066938345123237625.
K-9 Mean : 0.9597264437689971 with Standar Deviation : 0.009641776246542168.
K-10 Mean : 0.9595744680851063 with Standar Deviation : 0.009249315534773829.
K-11 Mean : 0.9598784194528877 with Standar Deviation : 0.008494339582056015.
